<a href="https://colab.research.google.com/github/nguyenhoanganh2002/Optimal-planning/blob/main/CBUS_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install arcpy

ERROR: Could not find a version that satisfies the requirement arcpy (from versions: none)
ERROR: No matching distribution found for arcpy


In [ ]:
import numpy as np
import scipy
from scipy import spatial
import arcpy
from arcpy import *
from arcpy.sa import *
import time
import math

start_time = time.time()

# Location of data
ObserverPoint = arcpy.GetParameterAsText(0)
arcpy.AddField_management(ObserverPoint, 'PctVisible', 'FLOAT')
FlightPath = arcpy.GetParameterAsText(1)
rasterDEM = arcpy.GetParameterAsText(2)

# Additional Parameters
FlightAltitude = arcpy.GetParameterAsText(3)
zFlight = float(FlightAltitude) * 0.3048  # elevation of airplane, in meters

mxd = arcpy.mapping.MapDocument('CURRENT')
df = arcpy.mapping.ListDataFrames(mxd)[0]

# find cell size of raster, assuming it's square
cellSizeProperty = GetRasterProperties_management(rasterDEM, 'CELLSIZEX')
cellSize = float(cellSizeProperty.getOutput(0))

# Determine the location of the observer point
obsvCursor = arcpy.UpdateCursor(ObserverPoint)
desc = arcpy.Describe(ObserverPoint)
shapefieldname = desc.ShapeFieldName

# XY location of observer on the ground
obsvIndex = 0
for obsv in obsvCursor:
    obsvFeature = obsv.getValue(shapefieldname)
    obsvPt = obsvFeature.getPart()

    # ArcPy euclidean distance tool
    rasterDistance = EucDistance(obsvFeature, "", cellSize)

    # array of data converted from raster to NumPy array
    x, y = np.mgrid[0:25:cellSize, 0:25:cellSize]
    arrayZ = arcpy.RasterToNumPyArray(rasterDEM)

    offsetXresult = GetRasterProperties_management(rasterDEM, 'LEFT')
    offsetX = float(offsetXresult.getOutput(0))

    offsetYresult = GetRasterProperties_management(rasterDEM, 'BOTTOM')
    offsetY = float(offsetYresult.getOutput(0))

    # calculate offset to shift geographic coordinates
    # to array locations
    arrayRowMax = arrayZ.shape[0] - 1

    obsvX = math.floor((obsvPt.X - float(offsetX)) / cellSize)
    obsvY = math.floor((obsvPt.Y - float(offsetY)) / cellSize)
    obsvCol = obsvX
    obsvRow = arrayRowMax - obsvY

    # elevation of the observer [z]
    zObserver = arrayZ[obsvRow, obsvCol]

    # Raster calculator, angle = arctan (z / d)
    arrayDistance = arcpy.RasterToNumPyArray(rasterDistance)
    arrayCellDistance = arrayDistance / cellSize
    arrayCorrectedZ = arrayZ - zObserver
    arrayTangent = arrayCorrectedZ / arrayCellDistance

    blockedSightLines = 0
    totalSightLines = 0
    obstructionList = []

    # Identify the geometry field
    desc = arcpy.Describe(FlightPath)
    shapefieldname = desc.ShapeFieldName

    # Create search cursor
    flightCursor = arcpy.SearchCursor(FlightPath)

    # Enter the for loop for each feature/row
    for flightPt in flightCursor:
        # Create the geometry object
        feat = flightPt.getValue(shapefieldname)
        # Set the start point
        startpt = feat.firstPoint

        # Set Start coordinates
        startX = math.floor((startpt.X - float(offsetX)) / cellSize)
        startCol = startX
        startY = math.floor((startpt.Y - float(offsetY)) / cellSize)
        startRow = arrayRowMax - startY

        # Set end point
        endpt = feat.lastPoint

        # Set End coordinates
        endX = math.floor((endpt.X - float(offsetX)) / cellSize)
        endCol = endX
        endY = math.floor((endpt.Y - float(offsetY)) / cellSize)
        endRow = arrayRowMax - endY

        zFP1 = arrayZ[startRow, startCol]
        zFP2 = arrayZ[endRow, endCol]

        # flight path begins at
        fpX0, fpY0 = startRow, startCol

        # flight path ends at
        fpX1, fpY1 = endRow, endCol
        arrayFlightPath = [[fpX0, fpY0], [fpX1, fpY1]]

        flightLength = scipy.spatial.distance.pdist(arrayFlightPath, 'euclidean')
        numFPts = int(flightLength)  # / int(cellSize)

        # Make a line with 'num' points distributed along it
        fpX, fpY = np.linspace(fpX0, fpX1, numFPts), np.linspace(fpY0, fpY1, numFPts)
        dzMax = zFlight - zObserver  # difference between flight altitude and observer

        for indexFP, point in np.ndenumerate(fpX):
            flightX = fpX[indexFP]
            flightY = fpY[indexFP]
            arraySightLine = [[obsvRow, obsvCol], [flightX, flightY]]

            sightLength = scipy.spatial.distance.pdist(arraySightLine, 'euclidean')
            numSPts = int(sightLength)
            slX, slY = np.linspace(obsvRow, flightX, numSPts), np.linspace(obsvCol, flightY, numSPts)

            # drop first values so observer point will not be evaluated
            slX = slX[1:]
            slY = slY[1:]
            dMax = sightLength
            tangentFlightPoint = dzMax / dMax  # tangent of observer sight line vertical angle

            tangentActual = arrayTangent[slX.astype(np.int), slY.astype(np.int)]
            listAngles = []
            blockedPoints = 0

            for indexSL, pointSL in np.ndenumerate(slX):
                sightX = slX[indexSL]
                sightY = slY[indexSL]
                arrayTempLine = [[obsvRow, obsvCol], [sightX, sightY]]
                tangentPoint = tangentActual[indexSL]
                listAngles.append(tangentPoint)

                if max(listAngles) > tangentFlightPoint:
                    blockedSightLines += 1
                totalSightLines += 1

    # time result
    arcpy.AddMessage(' >> analyzed in ' + str(time.time() - start_time) + ' seconds')
    obsvIndex = obsvIndex + 1
    percentVisible = 1. - (float(blockedSightLines) / float(totalSightLines))
    arcpy.AddMessage('========== observer point ' + str(obsvIndex) + ' analysis finished')
    arcpy.AddMessage('FINAL RESULT: flight path is ' + str(100 * percentVisible) + ' visible.')
    obsv.PctVisible = percentVisible
    obsvCursor.updateRow(obsv)

del obsv, obsvCursor, flightPt, flightCursor

# time result
arcpy.AddMessage('time elapsed: {} seconds'.format(str(time.time() - start_time)))

SyntaxError: ignored

In [ ]:
pip install ortools


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import time

In [ ]:
def fit(inv, dis_mat, n):
    summ = np.sum([dis_mat[int(inv[i])][int(inv[i+1])] for i in range(2*n-1)])
    return summ + dis_mat[0][int(inv[0])] + dis_mat[int(inv[-1])][0]

def input_from_text(path):
    f = open(path, "r")
    lines = f.readlines()
    n, k = int(lines[0].split()[0]), int(lines[0].split()[1])
    c = []
    for line in lines[1:-2]:
        c.append([float(c_i) for c_i in line.split()])
    f.close()
    return n, k, c, fit([int(c_i) for c_i in lines[-1].split()], dis_mat=c, n=n)

def Input():
    line1 = input().split()
    n, k = [int(i) for i in line1]
    c = []
    for i in range(2*n + 1):
        str_ci = input().split()
        c_i = [int(j) for j in str_ci]
        c.append(c_i)
    return n, k, c

def nOfBlindingConstrains(inv):
    count = 0
    mark = [0]*(n+1)
    n_passengers = 0
    for i in range(2*n):
        if inv[i] <= n:
            n_passengers += 1
            mark[inv[i]] = 1
        else:
            n_passengers -= 1
            mark[inv[i]-n] = 0
        if n_passengers > k:
            count += 1

    return count + np.sum(mark)

def generate_dataset(n, max_w, seed):
    random.seed(seed)
    dis_mat = [[0 for i in range(0, 2*n+1)] for j in range(0, 2*n+1)]
    for i in range(0, 2*n+1):
        for j in range(i+1, 2*n+1):
            dis_mat[i][j] = randint(1, max_w)
            dis_mat[j][i] = dis_mat[i][j]
    return dis_mat

In [ ]:
n, k, c, erp_ov = input_from_text("/content/drive/MyDrive/TULKH/n500_k40.txt")
erp_ov

6552.0

In [ ]:
# gerate random dataset
n, k = 250, 40
c = generate_dataset(n, 50, 1)

# **Branch and Bound**

In [ ]:
x = np.zeros(2*n + 1, dtype=int)        # [0, 0, 0,...0, 0]
visited = np.zeros(2*n + 1, dtype=bool) # [False, False,...False, False]
cities = list(np.arange(1, 2*n + 1))    # [1, 2, 3,...2n-1, 2n]
load = 0
best_fitness = 1e9
best_sol = None
d_min = c[0][1]

for i in range(2*n + 1):
    for j in range(2*n + 1):
        if ((c[i][j] < d_min) and (c[i][j] > 0)):
            d_min = c[i][j]

def fitness(j):
    res = c[x[j]][0]
    for idx in range(j):
        res += c[x[idx]][x[idx+1]]

    return res

def check(v):
    if visited[v] == True:
        return False

    # Trả khách
    if v > n:
        if visited[v-n] == True:
            return True
        return False
    # Đón khách
    else:
        if load < k:
            return True
        return False

def Try(j):
    global load
    global best_fitness
    global best_sol

    for v in cities:
        if check(v):
            x[j] = v

            # Cập nhật các ràng buộc cục bộ
            visited[v] = True
            if v <= n:  load += 1
            else:       load -= 1

            # Cập nhật kỷ lục
            if j == 2*n:
                c_fitness = fitness(j)
                if c_fitness < best_fitness:
                    best_fitness = c_fitness
                    best_sol = x.copy()
            else:
                if (fitness(j) - c[x[j]][0] + (2*n - j + 1) * d_min) < best_fitness:
                    Try(j + 1)

            # Khôi phục ràng buộc cục bộ
            visited[v] = False
            if v <= n:  load -= 1
            else:       load += 1
    #end
t1 = time.time()
Try(1)
print(f"Time: {time.time()-t1}")
print(f"Objective value: {best_fitness}")
print(f"Solution: {best_sol}")
print(f"Number of blinding constrains: {nOfBlindingConstrains(best_sol[1:])}")

# **Constrains Programming**

In [ ]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

x = []
for i in range(2*n + 1):
    x_i = []
    for j in range(2*n + 1):
        if i == j:
            x_i.append(0)
        else:
            x_i.append(model.NewIntVar(0, 1, f'x[{i}][{j}]'))
    x.append(x_i)

y = [model.NewIntVar(1, 2*n + 1, f'y[{i}]') for i in range(2*n + 1)]

p = [model.NewIntVar(0, k, f'p[{i}]') for i in range(2*n + 1)]

###### constrains ----------------------------------------------------------
# one way in, one way out
for i in range(2*n + 1):
    model.Add(sum(x[i][j] for j in range(2*n + 1)) == 1)
    model.Add(sum(x[j][i] for j in range(2*n + 1)) == 1)

# start at city 0
model.Add(y[0] == 1)

# n of passenger at city 0 equals 0
model.Add(p[0] == 0)

# not subcycles
for i in range(2*n + 1):
    for j in range(1, 2*n + 1):
        if i == j:
            continue
        model.Add(y[i]+1 == y[j]).OnlyEnforceIf(x[i][j])

# picks before drops
for i in range(n+1, 2*n + 1):
    model.Add(y[i] > y[i - n])

# maximum k passengers
for i in range(2*n + 1):
    for j in range(1, 2*n+1):
        if i == j:
            continue
        # picks
        if j <= n:
            model.Add(p[j]-p[i]==1).OnlyEnforceIf(x[i][j])
        # drops
        if j > n:
            model.Add(p[i]-p[j]==1).OnlyEnforceIf(x[i][j])

###### Objective -------------------------------------------------------
model.Minimize(sum(sum(x[i][j]*c[i][j] for j in range(2*n + 1)) for i in range (2*n + 1)))

###### Solve -----------------------------------------------------------
solver = cp_model.CpSolver()
t1 = time.time()
stt = solver.Solve(model)
print(f"Time: {time.time()-t1}")
if stt == cp_model.OPTIMAL:
    print(f"Objective value: {solver.ObjectiveValue()}")
    sol = {}
    for i in range(2*n + 1):
        sol[solver.Value(y[i])] = i
    sol = [sol[i+1] for i in range(2*n + 1)]
    print(f"Solution: {sol}")
    print(f"Number of blinding constrains: {nOfBlindingConstrains(sol[1:])}")

# **Mixed Interger Programing**

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("SCIP")

###### variables -----------------------------------------------------------
x = []
for i in range(2*n + 1):
    x_i = []
    for j in range(2*n + 1):
        if i == j:
            x_i.append(0)
        else:
            x_i.append(solver.IntVar(0, 1, f'x[{i}][{j}]'))
    x.append(x_i)

y = [solver.IntVar(1, 2*n + 1, f'y[{i}]') for i in range(2*n + 1)]

p = [solver.IntVar(0, k, f'p[{i}]') for i in range(2*n + 1)]

###### constrains ----------------------------------------------------------
# one way in, one way out
for i in range(2*n + 1):
    solver.Add(sum(x[i][j] for j in range(2*n + 1)) == 1)
    solver.Add(sum(x[j][i] for j in range(2*n + 1)) == 1)

# start at city 0
solver.Add(y[0] == 1)

# n of passenger at city 0 equals 0
solver.Add(p[0] == 0)

# not subcycles
for i in range(2*n + 1):
    for j in range(1, 2*n + 1):
        if i == j:
            continue
        solver.Add(y[i] - y[j] + (2*n + 1)*x[i][j] <= 2*n)

# picks before drops
for i in range(n+1, 2*n + 1):
    solver.Add(y[i] >= (y[i - n] + 1))

# maximum k passengers
for i in range(2*n + 1):
    for j in range(1, 2*n+1):
        if i == j:
            continue
        # picks
        if j <= n:
            solver.Add(p[j] - p[i] >= 1 - (k + 1)*(1 - x[i][j]))
            solver.Add(p[j] - p[i] <= 1 + (k - 1)*(1 - x[i][j]))
        # drops
        if j > n:
            solver.Add(p[i] - p[j] >= 1 - (k + 1)*(1 - x[i][j]))
            solver.Add(p[i] - p[j] <= 1 + (k - 1)*(1 - x[i][j]))

###### Objective -------------------------------------------------------
solver.Minimize(sum(sum(x[i][j]*c[i][j] for j in range(2*n + 1)) for i in range (2*n + 1)))

###### Solve -----------------------------------------------------------
t1 = time.time()
stt = solver.Solve()
print(f"Time: {time.time()-t1}")
if stt == pywraplp.Solver.OPTIMAL:
    print(f"Objective value: {int(solver.Objective().Value())}")
    sol = {}
    for i in range(2*n + 1):
        sol[y[i].solution_value()] = i
    sol = [sol[i+1] for i in range(2*n + 1)]
    print(f"Solution: {sol}")
    print(f"Number of blinding constrains: {nOfBlindingConstrains(sol[1:])}")

# **Greedy**


In [ ]:
solution = [] # visit sequence
solution.append(0)

distance = 0 # total travel distance (cost)

visited = []  # array to mark visited point
visited.append(1)
for i in range(2*n):
    visited.append(0)

# function to check if node is valid. Valid if with current solution, number of guest in bus is less than k and target node is not visited. Return True if valid, False if not valid
def is_valid(sol, node: int) -> bool:
    #check if node is visited
    if visited[node] == 1:
        return False

    #construc next solution
    next_sol = sol.copy()
    next_sol.append(node)
    #validate it
    mark = [0]*(n+1)
    n_passengers = 0
    for city in next_sol[1:]:
        if city <= n:
            n_passengers += 1
            mark[city] = 1
        else:
            n_passengers -= 1
            mark[city-n] = 0
        if n_passengers > k:
            # print(sol)
            return False
    if np.sum(mark) - n_passengers != 0:
        # print(sol)
        return False
    return True

def solver(sol):
    for _ in range(1, 2*n + 1):

        #get last_node
        last_node = sol[-1]

        #loop through all possible move from last_node
        possible_move = [] #(distance, node)

        for i in range(1,2*n+1):
            #call function to check if node is valid
            if is_valid(sol, i) == False:
                continue

            #calculate distance and push to possible_move list
            possible_move.append((c[last_node][i], i)) #(distance, node)

        #sort possible_move list by distance, ascending
        possible_move.sort(key=lambda x: x[0])

        #pick first element of possible_move list (shortest distance)
        next_node = possible_move[0][1]

        #update solution
        sol.append(next_node)

        #mark visited
        visited[next_node] = 1

        #update distance
        global distance
        distance += possible_move[0][0]

    #recursion
    return sol

t1 = time.time()
#start recursion
abc = solver(solution)
print(f"Time: {time.time()-t1}")
print("Objective value: ", distance + c[abc[-1]][0])
print("Solution: ", abc)
print(f"Number of blinding constrains: {nOfBlindingConstrains(abc[1:])}")

Time: 7.043705463409424
Objective value:  865
Solution:  [0, 22, 7, 106, 12, 47, 73, 9, 121, 19, 70, 21, 168, 23, 59, 101, 55, 11, 16, 32, 53, 126, 10, 18, 6, 56, 69, 54, 2, 26, 110, 123, 61, 1, 90, 112, 117, 3, 20, 130, 39, 351, 40, 261, 33, 373, 15, 273, 102, 257, 99, 269, 29, 272, 25, 319, 41, 320, 141, 291, 34, 275, 68, 340, 74, 251, 79, 276, 158, 367, 109, 282, 81, 262, 52, 256, 38, 318, 37, 408, 13, 266, 100, 279, 4, 259, 116, 297, 84, 253, 119, 288, 42, 270, 28, 362, 27, 290, 96, 287, 89, 252, 162, 305, 113, 260, 120, 284, 178, 356, 63, 339, 71, 418, 149, 391, 98, 313, 77, 324, 51, 268, 82, 363, 45, 352, 92, 366, 216, 369, 72, 295, 181, 263, 50, 265, 60, 303, 213, 254, 125, 375, 30, 278, 49, 327, 62, 311, 83, 299, 142, 304, 108, 309, 139, 333, 46, 321, 192, 359, 24, 292, 210, 289, 132, 331, 76, 271, 88, 296, 94, 310, 35, 274, 107, 389, 14, 302, 91, 264, 341, 86, 17, 312, 57, 348, 191, 380, 147, 334, 144, 463, 129, 342, 160, 280, 177, 332, 226, 300, 115, 346, 31, 466, 170, 336, 1

# **GREEDY +**

In [ ]:
sqrt_n = np.sqrt(n)

def weight_j(r, i):
    ln = -np.log(r+1e-5)
    if ln < 1e-4:
        return 0
    return np.power(ln, 1 + 50*np.sqrt(i/n))

def valid(sol, city, visited):
    #check if node is visited
    if visited[city] == 1:
        return False

    next_sol = sol.copy()
    next_sol.append(city)

    mark = [0]*(n+1)
    n_passengers = 0
    for _city in next_sol[1:]:
        if _city <= n:
            n_passengers += 1
            mark[_city] = 1
        else:
            n_passengers -= 1
            mark[_city-n] = 0
        if n_passengers > k:
            return False

    if np.sum(mark)-n_passengers != 0:
        return False
    return True

def solve():
    solution = []
    solution.append(0) # start at city 0
    fitness = 0 # objective value
    visited = [0]*(2*n+1)
    visited[0] = 1
    for _ in range(1, 2*n + 1):

        #current city
        c_city = solution[-1]

        weights = [] # probability distribution
        possible_move = []
        d_max = 0

        for city in range(1, 2*n+1):
            if valid(solution, city, visited) == False:
                continue
            d_i = c[c_city][city]
            possible_move.append([city, d_i]) #[city, distance]
            # if d_i > d_max:
            #     d_max = d_i

        possible_move = sorted(possible_move, key=lambda x: x[1])

        if len(possible_move) > 20:
            possible_move = possible_move[:20]

        for city in possible_move:
            if d_max < city[1]: d_max=city[1]

        # print(solution)
        # print(possible_move)
        # print(weights)

        if d_max == 0:
            next_city = possible_move[0]
        else:
            # cal weights
            for move in possible_move:
                r = move[1]/d_max
                weights.append(weight_j(r, _))

            if np.sum(weights) == 0:
                next_city = possible_move[0]
            else:
                # choices next city
                next_city = random.choices(possible_move, k=1, weights=weights)[0]

        #update solution
        solution.append(next_city[0])

        #mark visited
        visited[next_city[0]] = 1

        fitness += next_city[1]

    return solution, fitness

def solver():
    record = 10e9
    best_sol = None
    for seed in range(10):
        print("seed: ", seed)
        random.seed(seed)
        sol, fit = solve()
        print(fit + c[sol[-1]][0])
        if record > fit:
            record = fit
            best_sol = sol.copy()
    return best_sol, record

t1 = time.time()
best_sol, best_fitness = solver()
print(f"Time: {time.time()-t1}")

print("Objective value: ", best_fitness + c[best_sol[-1]][0])
print("Solution: ", best_sol)
print(f"Number of blinding constrains: {nOfBlindingConstrains(best_sol[1:])}")

seed:  0
864
seed:  1
853
seed:  2
820
seed:  3
897
seed:  4
828
seed:  5
920
seed:  6
834
seed:  7
837
seed:  8
839
seed:  9
839
Time: 74.42213439941406
Objective value:  834
Solution:  [0, 163, 198, 127, 9, 73, 133, 128, 199, 68, 239, 67, 149, 137, 145, 95, 399, 66, 207, 132, 55, 185, 42, 449, 110, 26, 224, 228, 98, 5, 8, 234, 36, 202, 150, 51, 188, 140, 457, 448, 191, 139, 173, 247, 497, 94, 236, 395, 101, 59, 23, 168, 91, 452, 181, 273, 16, 309, 81, 389, 233, 387, 20, 382, 84, 301, 111, 489, 125, 375, 166, 351, 292, 24, 203, 270, 97, 334, 22, 317, 435, 180, 423, 169, 400, 377, 105, 142, 138, 341, 32, 348, 223, 286, 225, 345, 108, 272, 162, 344, 62, 318, 453, 193, 170, 378, 34, 258, 122, 473, 31, 316, 113, 420, 39, 412, 419, 237, 53, 484, 69, 305, 196, 276, 478, 89, 57, 312, 1, 361, 475, 41, 281, 11, 443, 392, 4, 259, 240, 172, 46, 261, 255, 160, 78, 282, 109, 339, 80, 413, 48, 487, 44, 266, 52, 331, 249, 441, 184, 347, 215, 434, 254, 177, 446, 171, 126, 416, 430, 88, 120, 490, 60, 

# **GA with Tabu Search**

In [ ]:
dis_mat = c
# hàm tính tổng khoảng cách
def totalDistance(inv):
    summ = np.sum([dis_mat[int(inv[i])][int(inv[i+1])] for i in range(2*n-1)])
    return summ + dis_mat[0][int(inv[0])] + dis_mat[int(inv[-1])][0]

def nOfBlindingConstrains(inv):
    count = 0
    mark = [0]*(n+1)
    n_passengers = 0
    for i in range(2*n):
        if inv[i] <= n:
            n_passengers += 1
            mark[inv[i]] = 1
        else:
            n_passengers -= 1
            mark[inv[i]-n] = 0
        if n_passengers > k:
            count += 1

    return count + np.sum(mark)

def fitness(inv): # O(N)
    pen = totalDistance(np.arange(1, len(inv) + 1))
    distance = totalDistance(inv)
    return 5*pen*nOfBlindingConstrains(inv)/np.sqrt(distance) + distance

# khởi tạo quần thể ban đầu
def initPopulation(cities, size):
    population = []

    for i in range(size):
        invi = list(cities)
        random.shuffle(invi)
        distance = fitness(invi)
        population.append([distance, invi, 0])

    return population

# Toán tử lai 1 điểm cắt
#   father1: 2,3,5|1,4    father2: 1,3,5|4,2     điểm cắt: 3
# new_invi1: 1,3,5,2,4  new_invi2: 2,3,5,1,4
def singlePointCrossover(father1, father2):
    point = random.randint(0, len(father1) - 1)

    new_invi1 = father2[0:point]
    for invi in father1:
        if invi not in new_invi1:
            new_invi1.append(invi)

    new_invi2 = father1[0:point]
    for invi in father2:
        if invi not in new_invi2:
            new_invi2.append(invi)

    return new_invi1, new_invi2

# Toán tử lai 2 điểm cắt
#   father1: 6,2|4,3|1,5    father2: 6,1|3,5|4,2     điểm cắt: 2,4
# new_invi1: 6,2,3,5,4,1  new_invi2: 6,1,4,3,5,2
def multiPointCrossover(father1, father2): # O(N^2)
    points = sorted(random.sample(father1, 2))
    point1 = int(points[0]) - 1
    point2 = int(points[1]) - 1

    new_invi1 = father2[point1:point2]
    c1 = 0
    for invi in father1:
        if invi not in new_invi1:
            c1 += 1
            if c1 <= point1:
                new_invi1.insert(c1-1, invi)
            else:
                new_invi1.append(invi)


    new_invi2 = father1[point1:point2]
    c1 = 0
    for invi in father2:
        if invi not in new_invi2:
            c1 += 1
            if c1 <= point1:
                new_invi2.insert(c1-1, invi)
            else:
                new_invi2.append(invi)

    return new_invi1, new_invi2

def multiPointCrossover1(father1, father2):
    points = sorted(random.sample(father1, 2))
    point1 = int(points[0]) - 1
    point2 = int(points[1]) - 1

    new_invi1 = father2[point1:point2]
    c1 = 0
    c2 = 0
    for invi in father1:
        if invi not in new_invi1:
            c1 += 1
            if c1 <= point1:
                new_invi1.append(invi)
            else:
                new_invi1.insert(c2, invi)
                c2 += 1


    new_invi2 = father1[point1:point2]
    c1 = 0
    c2 = 0
    for invi in father2:
        if invi not in new_invi2:
            c1 += 1
            if c1 <= point1:
                new_invi2.append(invi)
            else:
                new_invi2.insert(c2, invi)
                c2 += 1

    return new_invi1, new_invi2

def nDiffCities(inv1, inv2): # O(N)
    return np.sum(np.array(inv1)!=np.array(inv2))

def chooseFather(population): # O(p*N)
    father1 = random.choice(population)[1]
    tabu = []
    max_diff = 0
    for invi in population:
        diff = nDiffCities(father1, invi[1])
        tabu.append([diff, invi[1]])
        if diff > max_diff:
            max_diff = diff
    threshold = max_diff/2
    nonTabuList = []
    for tabu_i in tabu:
        if tabu_i[0] < threshold:
            continue
        nonTabuList.append(tabu_i[1])
    if len(nonTabuList) == 0:
        return None, None
    return father1, random.choice(nonTabuList)

# đột biến: chọn ngẫu nhiên 2 thành phố rồi đổi chỗ cho nhau, thực hiện 2 lần
# 2,1,5,3,4 => 4,1,5,3,2 => 1,4,5,3,2
def mutation(father, lenCities=51): # O(N)
    new_invi = father.copy()
    for _ in range(5):
        points = random.sample(new_invi, 2)
        temp = new_invi[int(points[0]) - 1]
        new_invi[int(points[0]) - 1] = new_invi[int(points[1]) - 1]
        new_invi[int(points[1]) - 1] = temp

    return new_invi

def tournaments(old_population, new_invivduals, pop_size = 100): # O(plogp)
    merge = sorted(old_population + new_invivduals, key=lambda x: (x[0], -x[2]))
    best_fitness = merge[0][0]
    new_population = merge[:10]
    selected = []
    for inv in merge[10:]:
        if inv[0]/best_fitness > 0.75 or inv[2] < 50:
            selected.append(inv)

    new_population = new_population + random.choices(selected, k=pop_size-10)

    for inv in new_population:
        inv[2] += 1

    return new_population

def GA(
    population,
    lenCities = 51,
    pop_size = 100,
    num_of_generations = 2000,
    crossover_rate = 0.99,
    mutation_rate = 0.2,
    crossover_func=singlePointCrossover
):

    for _ in range(num_of_generations):
        new_inviduals = []
        # sinh pop_size cá thể mới
        n_i = int(pop_size/2)
        for _i in range(n_i):                               # O(g*[p/2*(p*N + N^2 + N) + plogp]) = O(g*p(p*N + N^2 + p*logp))

            father1, father2 = chooseFather(population)
            if father1 != None:
                # xảy ra quá trình lai ghép (crossover)
                if random.random() <= crossover_rate:
                    # thực hiện lai ghép 2 cá thể này
                    inv1, inv2 = crossover_func(father1, father2)
                    # xảy ra quá trình đột biến (mutation)
                    if random.random() <= mutation_rate:
                        inv1 = mutation(inv1, lenCities)
                        inv2 = mutation(inv2, lenCities)
                # không có lai ghép xảy ra
                else:
                    inv1, inv2 = father1, father2
            else:
                return sorted(population)
            new_inviduals.append([fitness(inv1), inv1, 0])
            new_inviduals.append([fitness(inv2), inv2, 0])

        population = tournaments(population, new_inviduals, pop_size)
        population = sorted(population)
        best_invi = population[0]

        if best_invi[2] > 20*(n**0.5):
            return population

        if _ % 100 == 0:
            print(f'Loop: {_},\tFitness: {best_invi[0]},\tDistance: {totalDistance(best_invi[1])},\tBlinding constrains: {nOfBlindingConstrains(best_invi[1])}')

    return population

total_distance = 0
total_time = 0
total_bCons = 0
for n_seed in range(10):
    random.seed(n_seed)
    cities = np.arange(1, 2*n+1)
    population = initPopulation(cities, size=100) # Khởi tạo quần thể đầu tiên
    t1 = time.time()
    pop = GA(population=population, crossover_func=multiPointCrossover, lenCities=len(cities))
    total_distance += totalDistance(pop[0][1])
    total_time += (time.time() - t1)
    total_bCons += nOfBlindingConstrains(pop[0][1])

print(f"Objective value: {total_distance/10}")
# print(f"Solution: {[0] + pop[0][1]}")
print(f"Number of blinding constrains: {total_bCons/10}")
print(f"Time: {total_time/10}")

Loop: 0,	Fitness: 3891.920710896017,	Distance: 1135,	Blinding constrains: 18
Loop: 100,	Fitness: 1510.8079348410972,	Distance: 1028,	Blinding constrains: 3
Loop: 200,	Fitness: 1103.750044563274,	Distance: 935,	Blinding constrains: 1
Loop: 300,	Fitness: 978.8913508921506,	Distance: 796,	Blinding constrains: 1
Loop: 400,	Fitness: 930.5573364596264,	Distance: 741,	Blinding constrains: 1
Loop: 500,	Fitness: 897.6130858924163,	Distance: 703,	Blinding constrains: 1
Loop: 600,	Fitness: 713.0,	Distance: 713,	Blinding constrains: 0
Loop: 700,	Fitness: 684.0,	Distance: 684,	Blinding constrains: 0
Loop: 800,	Fitness: 683.0,	Distance: 683,	Blinding constrains: 0
Loop: 900,	Fitness: 650.0,	Distance: 650,	Blinding constrains: 0
Loop: 1000,	Fitness: 638.0,	Distance: 638,	Blinding constrains: 0
Loop: 1100,	Fitness: 624.0,	Distance: 624,	Blinding constrains: 0
Loop: 1200,	Fitness: 612.0,	Distance: 612,	Blinding constrains: 0
Loop: 1300,	Fitness: 584.0,	Distance: 584,	Blinding constrains: 0


KeyboardInterrupt: ignored

In [ ]:
# def multiPointCrossover(father1, father2, n_points):
#     points = sorted(random.sample(father1, n_points))
#     length = len(father1)
#     if n_points%2 != 0:
#         points.append(length)
#         n_points += 1

#     new_inv1 = father2
#     new_inv2 = father1
#     cl1, cl2 = [], []
#     for idx in range(0, n_points, 2):
#         cl1 += father2[points[idx]:points[idx+1]]
#         cl2 += father1[points[idx]:points[idx+1]]

#     for idx in range(0, n_points, 2):
#         i1, i2 = points[idx], points[idx+1]
#         i = 0
#         for inv in father1:
#             if i == i2 - i1:
#                 break
#             if inv in cl1:
#                 new_inv1[i1+i] = inv
#                 cl1.remove(inv)
#                 i += 1
#     for idx in range(0, n_points, 2):
#         i1, i2 = points[idx], points[idx+1]
#         i = 0
#         for inv in father2:
#             if i == i2 - i1:
#                 break
#             if inv in cl2:
#                 new_inv2[i1+i] = inv
#                 cl2.remove(inv)
#                 i += 1

#     return new_inv1, new_inv2